reference : https://medium.com/@jiangan0808/retrieval-augmented-generation-rag-with-open-source-hugging-face-llms-using-langchain-bd618371be9d

In [113]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [114]:
loader = PyPDFDirectoryLoader("./PureUU/")

docs_before_split = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap  = 50,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

docs_after_split[0]

Document(metadata={'source': 'PureUU\\UU Nomor 11 Tahun 2008.pdf', 'page': 0}, page_content='PRESIDEN \nREPUBLIK INDONESIA \n \n \n \n \nUNDANG-UNDANG REPUBLIK INDONESIA  \nNOMOR 11 TAHUN 2008  \n \nTENTANG   \nINFORMASI DAN TRANSAKSI ELEKTRONIK  \n \n \n \nDENGAN RAHMAT TUHAN YANG MAHA ESA  \nPRESIDEN REPUBLIK INDONESIA,  \n \n \nMenimbang :   a.  bahwa  pembangunan  nasional  adalah  suatu  proses  yang  \nberkelanjutan   yang   harus   senantiasa   tanggap   terhadap  \nberbagai dinamika yang terjadi di masyarakat;   \nb. bahwa  globalisasi  informasi  telah  menempatkan  Indonesia')

In [115]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

print(f'Before split, there were {len(docs_before_split)} documents loaded, with average characters equal to {avg_char_before_split}.')
print(f'After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split} (average chunk length).')

Before split, there were 38 documents loaded, with average characters equal to 1925.
After split, there were 131 documents (chunks), with average characters equal to 558 (average chunk length).


In [116]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2", 
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}
)

c:\Users\Frederick\anaconda3\envs\EmbeddingLearning\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [117]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [-4.49483283e-03  4.63943146e-02  2.39182543e-03 -6.82217255e-02
 -4.91762161e-02  5.14068408e-03 -4.39534575e-04 -5.95147547e-04
 -1.49304979e-02  4.00025994e-02  8.51267204e-02  2.69969329e-02
 -6.17414378e-02 -7.23876581e-02  4.85156663e-02  8.53422284e-03
 -4.06529289e-03 -1.67988818e-02 -3.73626165e-02 -7.28238001e-02
  6.18899949e-02 -1.16680814e-02 -1.30743645e-02 -3.74527350e-02
 -2.46761478e-02 -1.95850916e-02  4.25968952e-02  5.12392353e-03
  8.27544332e-02 -2.83982251e-02 -4.22499403e-02  4.41240929e-02
  2.84396559e-02  3.78157049e-02 -7.57979229e-02  6.91364035e-02
 -4.14750203e-02  5.09713171e-03  6.79858848e-02  3.49299214e-03
  4.01313417e-02 -7.01914951e-02 -3.67621928e-02 -1.29194036e-01
  3.38713340e-02 -4.47076485e-02 -4.29368280e-02 -3.14693823e-02
 -2.70717889e-02 -3.88108082e-02 -1.17798708e-01  2.05702465e-02
 -3.38048525e-02 -4.33661975e-02  1.51255950e-02 -1.17782660e-01
 -6.12874608e-03  3.86152267e-02  1.26771023e-02 -4

In [118]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [119]:
query = """Alat bukti penyidikan, penuntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang ini adalah """  
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

(7) Penyidik Pegawai Negeri Sipil sebagaimana dimaksud pada  
ayat (1) berkoordinasi dengan Penyidik Pejabat Polisi Negara  Republik      Indonesia      memberitahukan      dimulainya  penyidikan dan menyampaikan hasilnya kepada penuntut  
umum.  
 
(8) D a l a m    r a n g k a    m e n g u n g k a p    t i n d a k    p i d a n a    I n f o r m a s i   
E l e k t r o n i k    d a n    T r a n s a k s i    E l e k t r o n i k ,    p e n y i d i k    d a p a t   berkerja sama dengan penyidik negara lain untuk berbagi  informasi dan alat bukti.  
 
 
 
Pasal 44  
 
Alat bukti penyidikan, penuntutan dan pemeriksaan di sidang  pengadilan   menurut   ketentuan   Undang-Undang   ini   adalah


In [120]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [121]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("indonlp/cendol-mt5-small-chat")
model = AutoModelForSeq2SeqLM.from_pretrained("indonlp/cendol-mt5-small-chat")

In [134]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="indonlp/cendol-mt5-small-chat", max_length=1000)

from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)
llm.invoke(query)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'Alat bukti penyidikan, penuntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang ini adalah sarana penyidikan, penuntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang Nomor 1 Tahun 2009 tentang Penyidikan, Penntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang ini adalah sarana penyidikan, penuntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang Nomor 1 Tahun 2009 tentang Penyidikan, Penntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang Nomor 1 Tahun 2009 tentang Penyidikan, Penntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang Nomor 1 Tahun 2009 tentang Penyidikan, Penntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang Nomor 1 Tahun 2009 tentang Penyidikan, Penntutan dan pemeriksaan di sidang pengadilan menurut ketentuan Undang-Undang Nomor 1 Tahun 2009 tentang Penyidikan, Pen'

In [135]:
prompt_template = """
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Ikuti aturan berikut:
1. Jika Anda tidak tahu jawabannya, jangan mencoba membuat jawaban. Cukup katakan "Saya tidak bisa menemukan jawaban akhir, tetapi Anda dapat memeriksa tautan berikut".
2. Jika Anda menemukan jawabannya, tuliskan dalam kalimat yang ringkas maksimal lima kalimat.

{context}

Pertanyaan: {question}

Jawaban yang membantu:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [136]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [140]:
result = retrievalQA.invoke({"query": "Apa yang dimaksud sistem elektronik?"})
print(result['result'])

Sistem elektronik
